In [19]:
import statistics
import time
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import skewnorm
import pandas as pd
# from matplotlib.collections import PatchCollection
# from matplotlib.patches import Rectangle
# from matplotlib.markers import MarkerStyle

## Import the logs from .CSV and create DataFrame

In [32]:
logs_df = pd.read_csv('defectRemediation_log_simulation_vPrototype.csv', index_col='ID')
# logs_df['Date'] = pd.to_datetime(logs_df['Date'], format='%Y-%d-%m').dt.date
# logs_df['Timestamp'] = pd.to_datetime(logs_df['Timestamp'], format='%H:%M:%S').dt.time
# logs_df['Date'] = pd.to_datetime(logs_df['Date'], format='%Y-%d-%m')
logs_df['Timestamp'] = pd.to_datetime(logs_df['Timestamp'], format='%Y-%d-%m %H:%M:%S')
logs_df

,Defect_ID,Control_Type,State,Date,Timestamp
ID,,,,,
1,1,ACC03,new,2025-16-05,2025-05-16 00:05:31
2,2,ACC17,new,2025-16-05,2025-05-16 00:12:42
3,3,ACC28,new,2025-16-05,2025-05-16 00:13:44
4,1,ACC03,assign,2025-16-05,2025-05-16 00:18:57
5,1,ACC03,in-progress,2025-16-05,2025-05-16 00:22:43
6,1,ACC03,closed,2025-16-05,2025-05-16 00:34:56
7,2,ACC17,assign,2025-16-05,2025-05-16 00:42:45
8,4,AUTH18,new,2025-16-05,2025-05-16 01:02:12
9,2,ACC17,in-progress,2025-16-05,2025-05-16 01:12:56


In [33]:
hours = []
for row in logs_df.iterrows():
    hours.append(logs_df.loc[row[0]]['Timestamp'].hour)
    # logs_df[row]
# logs_df['Timestamp']
logs_df['Hour'] = hours
logs_df

,Defect_ID,Control_Type,State,Date,Timestamp,Hour
ID,,,,,,
1,1,ACC03,new,2025-16-05,2025-05-16 00:05:31,0
2,2,ACC17,new,2025-16-05,2025-05-16 00:12:42,0
3,3,ACC28,new,2025-16-05,2025-05-16 00:13:44,0
4,1,ACC03,assign,2025-16-05,2025-05-16 00:18:57,0
5,1,ACC03,in-progress,2025-16-05,2025-05-16 00:22:43,0
6,1,ACC03,closed,2025-16-05,2025-05-16 00:34:56,0
7,2,ACC17,assign,2025-16-05,2025-05-16 00:42:45,0
8,4,AUTH18,new,2025-16-05,2025-05-16 01:02:12,1
9,2,ACC17,in-progress,2025-16-05,2025-05-16 01:12:56,1


### Build incoming defects distributions from logs

In [34]:
# master dictionary with tracked empirical data to build distributions
empirical_dict = {}
incoming_dict = {}
control_types = logs_df['Control_Type'].unique()
for control_type in control_types:
    empirical_dict[control_type] = {'incoming_per_hour': [],
                                    'delta_new_assign': [],
                                    'delta_assign_inprogress': [],
                                    'delta_inprogress_closed': []
                                   }
    incoming_dict[control_type] = {}

In [26]:
incoming_dict = {}
sub_logs_df = logs_df.query('(Control_Type == "ACC03") and (State == "new")')
testing = sub_logs_df.query('Date == "2025-16-05"')['Hour'].value_counts()
testing
# testing.loc[18]
# sub_logs_df['Hour'].value_counts()

Hour
0    1
2    1
Name: count, dtype: int64

In [13]:
incoming_defects = np.zeros(24)
incoming_defects = [testing.loc[i] if i in testing.index else 0 for i in range(0,24)]
incoming_defects

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0]

In [35]:
# hours = list(range(0,24)) + [0] # list of hours from midnight to midnight
# hours = list(range(0,24))

# count no° of incoming defects per control type every hour per day WHERE STATE == NEW
# for day in logs_df["Date"].unique():
# sub_logs_df = logs_df.query('(Date == @day) and (State == "new")')
for control_type in control_types:
    sub_logs_df = logs_df.query('(Control_Type == @control_type) and (State == "new")')
    for day in logs_df["Date"].unique():
        sub_logs_df_date = sub_logs_df.query('Date == @day')['Hour'].value_counts()
        incoming_defects = [sub_logs_df_date.loc[i] if i in sub_logs_df_date.index else 0 for i in range(0,24)]
        incoming_dict[control_type].update({day: incoming_defects})
        empirical_dict[control_type]['incoming_per_hour'] = empirical_dict[control_type]['incoming_per_hour'] + incoming_dict[control_type][day]
    
# incoming_dict
len(empirical_dict['ACC03']['incoming_per_hour'])

96

## Creating Delta table to track time b/w state changes

#### If Delta table already exists, skip the next block; otherwise initialize the empty Delta table:

In [36]:
deltas_df = pd.DataFrame(columns=['Defect_ID','Control_Type','Delta_New_Assign','Delta_Assign_InProgress','Delta_InProgress_Closed'])

In [37]:
try: # deltas_df contains some entries, add onto it
    Last_ID_processed = deltas_df.index[-1]
    index = Last_ID_processed + 1
except IndexError: # deltas_df is empty, start from beginning
    index = 0 

for defect_id in logs_df['Defect_ID'].unique():
    #### Recover timestamps for each state change ####
    timestamp_new = logs_df.loc[(logs_df['Defect_ID']==defect_id) & (logs_df['State']=='new')]['Timestamp'].iloc[0] # OR logs_df.query('(Defect_ID == defect_id) and (State == "new")')['Timestamp'].iloc[0]
    timestamp_assign = logs_df.loc[(logs_df['Defect_ID']==defect_id) & (logs_df['State']=='assign')]['Timestamp'].iloc[0]
    timestamp_inprogress = logs_df.loc[(logs_df['Defect_ID']==defect_id) & (logs_df['State']=='in-progress')]['Timestamp'].iloc[0]
    timestamp_closed = logs_df.loc[(logs_df['Defect_ID']==defect_id) & (logs_df['State']=='closed')]['Timestamp'].iloc[0]

    # time between assign and new (hrs)
    delta_new_assign = timestamp_assign - timestamp_new
    delta_new_assign = round(delta_new_assign.total_seconds()/3600, 3)
    
    # time between in-progress and assign (hrs)
    delta_assign_inprogress = timestamp_inprogress - timestamp_assign
    delta_assign_inprogress = round(delta_assign_inprogress.total_seconds()/3600, 3)
    
    # time between closed and in-progress (hrs)
    delta_inprogress_closed = timestamp_closed - timestamp_inprogress
    delta_inprogress_closed = round(delta_inprogress_closed.total_seconds()/3600, 3)

    # append row to df and iterate index
    control_type = logs_df.loc[logs_df['Defect_ID'] == defect_id].iloc[0]['Control_Type']
    deltas_df.loc[index] = [defect_id, control_type, delta_new_assign, delta_assign_inprogress, delta_inprogress_closed]
    index += 1


In [38]:
deltas_df

,Defect_ID,Control_Type,Delta_New_Assign,Delta_Assign_InProgress,Delta_InProgress_Closed
0,1,ACC03,0.224,0.063,0.204
1,2,ACC17,0.501,0.503,0.779
2,3,ACC28,1.500,1.594,1.067
3,4,AUTH18,1.039,0.229,1.603
4,5,ACC03,0.845,0.466,0.731
5,6,AUTH42,0.378,0.029,0.092
6,7,ACC17,1.597,0.052,0.047
7,8,ACC03,0.329,0.206,2.983
8,9,ACC03,0.271,3.382,4.347
9,10,ACC03,1.506,2.044,2.186


In [39]:
deltas_df.groupby('Control_Type')[['Delta_New_Assign']].mean()

,Delta_New_Assign
Control_Type,
ACC03,7.912
ACC17,1.049
ACC28,1.500
AUTH18,1.039
AUTH42,0.378


### Build time b/w state changes distributions from Delta table

In [40]:
deltas_df

,Defect_ID,Control_Type,Delta_New_Assign,Delta_Assign_InProgress,Delta_InProgress_Closed
0,1,ACC03,0.224,0.063,0.204
1,2,ACC17,0.501,0.503,0.779
2,3,ACC28,1.500,1.594,1.067
3,4,AUTH18,1.039,0.229,1.603
4,5,ACC03,0.845,0.466,0.731
5,6,AUTH42,0.378,0.029,0.092
6,7,ACC17,1.597,0.052,0.047
7,8,ACC03,0.329,0.206,2.983
8,9,ACC03,0.271,3.382,4.347
9,10,ACC03,1.506,2.044,2.186


In [41]:
for control_type in control_types:
    empirical_dict[control_type]['delta_new_assign'].extend(deltas_df.query('Control_Type == @control_type')['Delta_New_Assign'].to_list())
    empirical_dict[control_type]['delta_assign_inprogress'].extend(deltas_df.query('Control_Type == @control_type')['Delta_Assign_InProgress'].to_list())
    empirical_dict[control_type]['delta_inprogress_closed'].extend(deltas_df.query('Control_Type == @control_type')['Delta_InProgress_Closed'].to_list())

In [42]:
empirical_dict

{'ACC03': {'incoming_per_hour': [1,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   2,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   4,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'delta_new_assign': [0.224,
   0.845,
   0.329,
   0.271,
   1.506,
   16.184,
   15.869,
   17.233,
   18.747],
  'delta_assign_inprogress': [0.063,
   0.466,
   0.206,
   3.382,
   2.044,
   3.04,
   0.028,
   1.986,
   1.334],
  'delta_inprogress_closed': [0.204,
   0.731,
   2.983,
   4.347,
   2.186,
   4.576,
   3.781,
   4.589,
   3.745]},
 'ACC17': {'incoming_per_hour': [1,
   0,


#### Create histograms from empirical_dict